* https://towardsdatascience.com/sentiment-analysis-in-10-minutes-with-bert-and-hugging-face-294e8a04b671
* https://arxiv.org/pdf/1810.04805.pdf
* https://huggingface.co/bert-base-uncased?text=Paris+is+the+%5BMASK%5D+of+France.
* https://huggingface.co/docs/transformers/main_classes/tokenizer
* https://www.tensorflow.org/api_docs/python/tf/keras/Model
* https://www.kaggle.com/datasets/yasserh/twitter-tweets-sentiment-dataset
* https://www.tensorflow.org/text/tutorials/classify_text_with_bert
* https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/classify_text_with_bert.ipynb#scrollTo=VBWzH6exlCPS
* https://www.tensorflow.org/api_docs/python/tf/keras/Model
* https://arxiv.org/pdf/1810.04805.pdf
* https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
* https://huggingface.co/docs/transformers/index
* https://www.kaggle.com/datasets/yasserh/imdb-movie-ratings-sentiment-analysis?select=movie.csv
* https://www.kaggle.com/datasets/ishantjuyal/emotions-in-text

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
from google.colab import files

## Install Transformers library

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.1 MB/s 
     |████████████████████████████████| 6.6 MB 51.1 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 62.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Load the BERT Classifier and Tokenizer along with Input modules

In [4]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
from transformers import BertConfig, BertModel

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [5]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  4614      
                                                                 
Total params: 109,486,854
Trainable params: 109,486,854
Non-trainable params: 0
_________________________________________________________________


## Download Kaggle dataset

In [6]:
! pip install kaggle
! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
! cp /content/gdrive/MyDrive/Colab\ Notebooks/Thesis/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
! kaggle datasets download -d ishantjuyal/emotions-in-text
! unzip emotions-in-text.zip

  0% 0.00/781k [00:00<?, ?B/s]
100% 781k/781k [00:00<00:00, 146MB/s]
Archive:  emotions-in-text.zip
  inflating: Emotion_final.csv       


In [8]:
path = '/content/Emotion_final.csv'
dataset_file = pd.read_csv(path)

In [9]:
dataset_file.head()

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [10]:
print(dataset_file.Emotion.unique())

['sadness' 'anger' 'love' 'surprise' 'fear' 'happy']


In [11]:
dataset_file.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
dataset_file['LABEL_COLUMN'] = dataset_file['LABEL_COLUMN'].replace('sadness', 0)
dataset_file['LABEL_COLUMN'] = dataset_file['LABEL_COLUMN'].replace('anger', 1)
dataset_file['LABEL_COLUMN'] = dataset_file['LABEL_COLUMN'].replace('love', 2)
dataset_file['LABEL_COLUMN'] = dataset_file['LABEL_COLUMN'].replace('surprise', 3)
dataset_file['LABEL_COLUMN'] = dataset_file['LABEL_COLUMN'].replace('fear', 4)
dataset_file['LABEL_COLUMN'] = dataset_file['LABEL_COLUMN'].replace('happy', 5)
dataset_file

,DATA_COLUMN,LABEL_COLUMN
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1
...,...,...
21454,Melissa stared at her friend in dism,4
21455,Successive state elections have seen the gover...,4
21456,Vincent was irritated but not dismay,4
21457,Kendall-Hume turned back to face the dismayed ...,4


## Split train sets

In [12]:
train, test_and_validatition = train_test_split(dataset_file, test_size=0.2, random_state=77)
# train_1, train_rest = train_test_split(train, test_size=0.5, random_state=77)
# train_2, train_3 = train_test_split(train_rest, test_size=0.5, random_state=77)
test, validation = train_test_split(test_and_validatition, test_size=0.5, random_state=77)
print(len(train),len(test),len(validation))

17167 2146 2146


## Save test and validation datasets

In [13]:
model_save_name = 'BERTModel-Emotions'
path = F"/content/gdrive/MyDrive/Colab_Notebooks/savedModels/{model_save_name}" 
testSetPath = F"/content/gdrive/MyDrive/Colab_Notebooks/savedModels/{model_save_name}-test.csv"
validationSetPath = F"/content/gdrive/MyDrive/Colab_Notebooks/savedModels/{model_save_name}-validation.csv"

In [14]:
# test.to_csv('{model_save_name}-test.csv', encoding = 'utf-8-sig') 
# files.download('{model_save_name}-test.csv')

# validation.to_csv('{model_save_name}-validation.csv', encoding = 'utf-8-sig') 
# files.download('{model_save_name}-validation.csv')


with open(testSetPath, 'w', encoding = 'utf-8-sig') as f:
  test.to_csv(f)

with open(validationSetPath, 'w', encoding = 'utf-8-sig') as f:
  validation.to_csv(f)

## Create input sequences

In [15]:
def convert_data_to_examples_single(inputDataset, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = inputDataset.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)  
  return train_InputExamples


def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [16]:
DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [17]:
test_inputExamples = convert_data_to_examples_single(test, DATA_COLUMN, LABEL_COLUMN)
test_data = convert_examples_to_tf_dataset(list(test_inputExamples), tokenizer)
test_data = test_data.batch(32)

validation_InputExamples = convert_data_to_examples_single(validation, DATA_COLUMN, LABEL_COLUMN)
validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


## Configure the Loaded BERT model and Train for Fine-tuning

In [18]:
train_InputExamples  = convert_data_to_examples_single(train, DATA_COLUMN, LABEL_COLUMN)
train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [19]:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=3e-5, epsilon=1e-08), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
1074/1074 [==============================] - 896s 803ms/step - loss: 0.3276 - accuracy: 0.8824 - val_loss: 0.1809 - val_accuracy: 0.9315
Epoch 2/2
1074/1074 [==============================] - 859s 800ms/step - loss: 0.0950 - accuracy: 0.9566 - val_loss: 0.1939 - val_accuracy: 0.9301


In [20]:
model.evaluate(test_data)

68/68 [==============================] - 19s 278ms/step - loss: 0.2111 - accuracy: 0.9268


[0.21111895143985748, 0.9268406629562378]

In [21]:
model.save_pretrained(path)

## Load model

In [22]:
loaded_model = TFBertForSequenceClassification.from_pretrained(path, local_files_only=True)
loaded_model.summary()

Some layers from the model checkpoint at /content/gdrive/MyDrive/Colab_Notebooks/savedModels/BERTModel-Emotions were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/gdrive/MyDrive/Colab_Notebooks/savedModels/BERTModel-Emotions.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predict

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  4614      
                                                                 
Total params: 109,486,854
Trainable params: 109,486,854
Non-trainable params: 0
_________________________________________________________________


## Make Predictions with the Fine-tuned Model

In [23]:
pred_sentences = ['I am scared of the dark',
                  'I want to spend the rest of my life with you',
                  'He was filled with joy when he opened his present',
                  'She was devastated after the death of her husband']

In [24]:
tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = loaded_model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Sadness', 'Anger', 'Love', 'Surprise', 'Fear', 'Happiness']
label = tf.argmax(tf_predictions, axis=1)
label = label.numpy()
for i in range(len(pred_sentences)):
  print(pred_sentences[i], ": \n", labels[label[i]])

I am scared of the dark : 
 Fear
I want to spend the rest of my life with you : 
 Love
He was filled with joy when he opened his present : 
 Happiness
She was devastated after the death of her husband : 
 Sadness
